## Separating Frontscan and Backscan for different number of Sensors

The key ideas here are:

- Functions like moduleAnalysis() returns two identically structured dictionaries that contain the keys like xstart, ystart, zstart, xinc, yinc, zinc, Nx, Ny, Nz, orientation. For the function arguments like sensorsy or sensorsx, there is an assumption that those will be equal for both the front and back surface.

- We need to develop a separate function, pretty much functionally parallel with moduleAnalysis() to bring out the frontscan and backscan separately.....may be two distinct functions with distinct arguments for frontscan() and backscan()

- The new functions will have variables passed on as arguments which can be different for front and back

In [1]:
import bifacial_radiance
import numpy as np
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')

In [2]:
demo = bifacial_radiance.RadianceObj('ScanSeparate', testfolder) 

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


In [3]:
x = 2
y = 1
xgap = 0.02
ygap = 0.15
zgap = 0.10
numpanels = 1
offsetfromaxis = True
axisofrotationTorqueTube = True
frameParams = None
omegaParams = None
diam = 0.1

In [4]:
module_type = 'TEST'
nMods = 3
nRows = 2
sceneDict = {'tilt':30, 'pitch':6, 'clearance_height':3,'azimuth':90, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo.setGround(0.2)
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)
metdata = demo.readWeatherFile(epwfile, coerce_year = 2021)
demo.gendaylit(4020)

Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
start: 2021-01-01 01:00:00
end: 2021-12-31 23:00:00
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


'skies\\sky2_37.5_-77.33_2021_06_17_13_00.rad'

In [6]:
demo.makeModule(name=module_type,x=x, y=y, torquetube = True, 
                    diameter = diam, xgap = xgap, ygap = ygap, zgap = zgap, 
                    numpanels = numpanels, omegaParams=None,
                    axisofrotationTorqueTube=axisofrotationTorqueTube)


Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Module TEST updated in module.json


{'x': 2,
 'y': 1,
 'z': 0.02,
 'modulematerial': 'black',
 'scenex': 2.02,
 'sceney': 1.0,
 'scenez': 0.15,
 'numpanels': 1,
 'bifi': 1,
 'text': '! genbox black TEST 2 1 0.02 | xform -t -1.0 -0.5 0.15 -a 1 -t 0 1.15 0\r\n! genrev Metal_Grey tube1 t*2.02 0.05 32 | xform -ry 90 -t -1.01 0 0',
 'modulefile': 'objects\\TEST.rad',
 'offsetfromaxis': 0.15,
 'xgap': 0.02,
 'ygap': 0.15,
 'zgap': 0.1,
 'cellModule': None,
 'torquetube': {'bool': True,
  'diameter': 0.1,
  'tubetype': 'Round',
  'material': 'Metal_Grey'}}

In [7]:
scene = demo.makeScene(module_type,sceneDict)
octfile = demo.makeOct()
analysis = bifacial_radiance.AnalysisObj()  # return an analysis object including the scan dimensions for back irradiance

Created ScanSeparate.oct


In [8]:
name = 'ModuleScanTest'
rowWanted = 1
modWanted = 2


In [9]:

sensors_diff = True


# Test 0 - Same sensors Front and back, xsensors=y

In [10]:
name='test0'

sensorsy = 2
sensorsy_back = 2

sensorsx = 1
sensorsx_back = 1

frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

print('Start shift:', start_shift, 'flag:', flag_s)
print('frontscan', frontscan)
print('backscan', backscan, '\n')

frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, 
                                        backscan = backscan, start_shift = start_shift, flag_s = flag_s)
print('\nBOTH\n', bifacial_radiance.load.read1Result('results\irr_'+name+'.csv'))


Start shift: None flag: None
frontscan {'xstart': 0.22983756729740645, 'ystart': 1.4073492055729158e-17, 'zstart': 3.2397570107138054, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 1, 'Ny': 2.0, 'Nz': 1, 'orient': '-0.500 -0.000 -0.866'}
backscan {'xstart': 0.21883756729740644, 'ystart': 1.3399936316198115e-17, 'zstart': 3.2207044518305477, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 1, 'Ny': 2, 'Nz': 1, 'orient': '0.500 0.000 0.866'} 

Linescan in process: test0_Front
Linescan in process: test0_Back
Saved: results\irr_test0.csv

BOTH
           x             y         z     rearZ            mattype  \
0  0.229838  1.407349e-17  3.239757  3.220704  a1.0.a0.TEST.6457   
1 -0.058838 -3.602762e-18  3.406424  3.387371  a1.0.a0.TEST.6457   

             rearMat  Wm2Front   Wm2Back  Back/FrontRatio  
0  a1.0.a0.TEST.2310  777.0739  146.4997         0.188527  
1  a1.0.a0.TEST.2310  777.2

# Test 1: more sensors on back than front, xsensors=1

In [11]:
name='test1'

sensorsy = 2
sensorsy_back = 4

sensorsx = 1
sensorsx_back = 1


frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

print('Start shift:', start_shift, 'flag:', flag_s)
print('frontscan', frontscan)
print('backscan', backscan, '\n')

frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, 
                                        backscan = backscan, start_shift = start_shift, flag_s = flag_s)
print('\nFRONT\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Front.csv'))
print('BACK\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Back.csv'))

Start shift: None flag: None
frontscan {'xstart': 0.22983756729740645, 'ystart': 1.4073492055729158e-17, 'zstart': 3.2397570107138054, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 1, 'Ny': 2.0, 'Nz': 1, 'orient': '-0.500 -0.000 -0.866'}
backscan {'xstart': 0.3343076211353315, 'ystart': 2.0470437907697492e-17, 'zstart': 3.1540377851638812, 'xinc': -0.17320508075688776, 'yinc': -1.0605752387249071e-17, 'zinc': 0.09999999999999999, 'Nx': 1, 'Ny': 4, 'Nz': 1, 'orient': '0.500 0.000 0.866'} 

Linescan in process: test1_Front
Linescan in process: test1_Back
Saved: results\irr_test1_Front.csv
Saved: results\irr_test1_Back.csv

FRONT
           x             y         z            mattype       Wm2
0  0.229838  1.407349e-17  3.239757  a1.0.a0.TEST.6457  777.5776
1 -0.058838 -3.602762e-18  3.406424  a1.0.a0.TEST.6457  777.6735
BACK
           x             y         z            mattype         Wm2
0  0.334308  2.047044e-17  3.154038  a1.0.a0

# Test 2 - Same sensors front and back, xsensors = 2

In [12]:
name='test2'

sensorsy = 2
sensorsy_back = 2

sensorsx = 2
sensorsx_back = 2


frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

print('Start shift:', start_shift, 'flag:', flag_s)
print('frontscan', frontscan)
print('backscan', backscan, '\n')

frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, 
                                        backscan = backscan, start_shift = start_shift, flag_s = flag_s)
print('\nBOTH\n', bifacial_radiance.load.read1Result('results\irr_'+name+'.csv'))


Start shift: -0.6666666666666666 flag: 0
frontscan {'xstart': 0.22983756729740645, 'ystart': 0.33333333333333337, 'zstart': 3.2397570107138054, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 2, 'Ny': 2.0, 'Nz': 1, 'orient': '-0.500 -0.000 -0.866'}
backscan {'xstart': 0.21883756729740644, 'ystart': 0.33333333333333337, 'zstart': 3.2207044518305477, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 2, 'Ny': 2, 'Nz': 1, 'orient': '0.500 0.000 0.866'} 

Linescan in process: test2_Front
Linescan in process: test2_Back
Saved: results\irr_test2.csv

BOTH
           x         y         z     rearZ            mattype  \
0  0.229838  0.333333  3.239757  3.220704  a1.0.a0.TEST.6457   
1 -0.058838  0.333333  3.406424  3.387371  a1.0.a0.TEST.6457   
2 -0.436829  0.333333  3.239757  3.220704        groundplane   
3 -0.725504  0.333333  3.406424  3.387371        groundplane   

             rearMat    W

# Test 3 - Different sensors front and back, xsensors = 2

In [13]:
name='test3'

sensorsy = 2
sensorsy_back = 4

sensorsx = 2
sensorsx_back = 2

frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

print('Start shift:', start_shift, 'flag:', flag_s)
print('frontscan', frontscan)
print('backscan', backscan, '\n')

frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, 
                                        backscan = backscan, start_shift = start_shift, flag_s = flag_s)
print('\nFRONT\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Front.csv'))
print('BACK\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Back.csv'))

Start shift: -0.6666666666666666 flag: 0
frontscan {'xstart': 0.22983756729740645, 'ystart': 0.33333333333333337, 'zstart': 3.2397570107138054, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 2, 'Ny': 2.0, 'Nz': 1, 'orient': '-0.500 -0.000 -0.866'}
backscan {'xstart': 0.3343076211353315, 'ystart': 0.33333333333333337, 'zstart': 3.1540377851638812, 'xinc': -0.17320508075688776, 'yinc': -1.0605752387249071e-17, 'zinc': 0.09999999999999999, 'Nx': 2, 'Ny': 4, 'Nz': 1, 'orient': '0.500 0.000 0.866'} 

Linescan in process: test3_Front
Linescan in process: test3_Back
Saved: results\irr_test3_Front.csv
Saved: results\irr_test3_Back.csv

FRONT
           x         y         z            mattype         Wm2
0  0.229838  0.333333  3.239757  a1.0.a0.TEST.6457  777.482400
1 -0.058838  0.333333  3.406424  a1.0.a0.TEST.6457  777.579800
2 -0.436829  0.333333  3.239757        groundplane  892.676367
3 -0.725504  0.333333  3.406424        groundplane  89

# Test 4 - Different sensors front and back, different xsensors

In [14]:
name='test4'

sensorsy = 2
sensorsy_back = 4

sensorsx = 2
sensorsx_back = 4

frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

print('Start shift:', start_shift, 'flag:', flag_s)
print('frontscan', frontscan)
print('backscan', backscan, '\n')

frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, 
                                        backscan = backscan, start_shift = start_shift, flag_s = flag_s)
print('\nFRONT\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Front.csv'))
print('BACK\n', bifacial_radiance.load.read1Result('results\irr_'+name+'_Back.csv'))

Start shift: -0.6666666666666666 flag: 0
frontscan {'xstart': 0.22983756729740645, 'ystart': 0.33333333333333337, 'zstart': 3.2397570107138054, 'xinc': -0.28867513459481287, 'yinc': -1.7676253978748448e-17, 'zinc': 0.16666666666666663, 'Nx': 2, 'Ny': 2.0, 'Nz': 1, 'orient': '-0.500 -0.000 -0.866'}
backscan {'xstart': 0.3343076211353315, 'ystart': 0.6, 'zstart': 3.1540377851638812, 'xinc': -0.17320508075688776, 'yinc': -1.0605752387249071e-17, 'zinc': 0.09999999999999999, 'Nx': 4, 'Ny': 4, 'Nz': 1, 'orient': '0.500 0.000 0.866'} 

Linescan in process: test4_Front
Linescan in process: test4_Back
Saved: results\irr_test4_Front.csv
Saved: results\irr_test4_Back.csv

FRONT
           x         y         z            mattype         Wm2
0  0.229838  0.333333  3.239757  a1.0.a0.TEST.6457  777.645000
1 -0.058838  0.333333  3.406424  a1.0.a0.TEST.6457  777.770300
2 -0.436829  0.333333  3.239757        groundplane  893.056000
3 -0.725504  0.333333  3.406424        groundplane  894.835733
BACK
  